# My First Query

One of the most powerful features of Marvin 2.0 is ability to query the newly created DRP and DAP databases. You can do this in two ways:
1. via the **[Marvin-web Search page](https://sas.sdss.org/marvin2/search/)** or
2. via Python (in the terminal/notebook/script) with **Marvin-tools**.

The best part is that both interfaces use the same underlying query structure, so your input search will be the same. Here we will run a few queries with Marvin-tools to learn the basics of how to construct a query and also test drive some of the more advanced features that are unique to the Marvin-tools version of querying.

In [37]:
# Python 2/3 compatibility
from __future__ import print_function, division, absolute_import

In [38]:
from marvin import config
config.mode = 'remote'

In [39]:
from marvin.tools.query import Query

Let's search for galaxies with M$_\star$ > 3 $\times$ 10$^{11}$ M$_\odot$.

To specify our search parameter, M$_\star$, we must know the database table and name of the parameter. In this case, MaNGA uses the NASA-Sloan Atlas (NSA) for target selection so we will use the Sersic profile determination for stellar mass, which is the **`sersic_mass`** parameter of the **`nsa`** table, so our search parameter will be **`nsa.sersic_mass`**.

Generically, the search parameter will take the form **`table.parameter`**.

In [40]:
myquery1 = 'nsa.sersic_mass > 3e11'

In [44]:
q1 = Query(searchfilter=myquery1)
print(q1)
r1 = q1.run()
from marvin.tools.query import doQuery
q, r = doQuery(searchfilter=myquery1)
print(r.totalcount)
r1.showQuery()

Your parsed filter is: 
nsa.sersic_mass>3e11
Query(mode='remote', limit=10, sort=None, order='asc')
Results contain of a total of 7, only returning the first 7 results
Your parsed filter is: 
nsa.sersic_mass>3e11
Results contain of a total of 7, only returning the first 7 results
7


u'SELECT mangadatadb.cube.mangaid, mangadatadb.cube.plate, mangadatadb.ifudesign.name, mangasampledb.nsa.sersic_mass \nFROM mangadatadb.cube JOIN mangadatadb.ifudesign ON mangadatadb.ifudesign.pk = mangadatadb.cube.ifudesign_pk JOIN mangasampledb.manga_target ON mangasampledb.manga_target.pk = mangadatadb.cube.manga_target_pk JOIN mangasampledb.manga_target_to_nsa ON mangasampledb.manga_target.pk = mangasampledb.manga_target_to_nsa.manga_target_pk JOIN mangasampledb.nsa ON mangasampledb.nsa.pk = mangasampledb.manga_target_to_nsa.nsa_pk JOIN mangadatadb.pipeline_info AS drpalias ON drpalias.pk = mangadatadb.cube.pipeline_info_pk \nWHERE mangasampledb.nsa.sersic_mass > 3e+11 AND drpalias.pk = 21'

Running the query produces a **`Results`** object (**`r1`**):

In [42]:
# show results
print('total count', r1.totalcount)
r1.results

total count 7


[NamedTuple(mangaid=u'1-377378', plate=8132, name=u'6102', sersic_mass=513574535168.0),
 NamedTuple(mangaid=u'1-377841', plate=8133, name=u'12705', sersic_mass=317070770176.0),
 NamedTuple(mangaid=u'1-338090', plate=8131, name=u'3701', sersic_mass=1150581604352.0),
 NamedTuple(mangaid=u'1-217041', plate=8247, name=u'9102', sersic_mass=328593276928.0),
 NamedTuple(mangaid=u'1-134760', plate=8555, name=u'12701', sersic_mass=561190207488.0),
 NamedTuple(mangaid=u'1-256301', plate=8258, name=u'9102', sersic_mass=7116798033920.0),
 NamedTuple(mangaid=u'1-278075', plate=8257, name=u'6104', sersic_mass=369127456768.0)]

We will learn how to use the features of our **`Results`** object a little bit later, but first let's revise our search to see how more complex search queries work.

## Multiple Search Criteria

Let's add to our search to find only galaxies with a redshift less than 0.1.

Redshift is the **`z`** parameter and is also in the **`nsa`** table, so its full search parameter designation is **`nsa.z`**.

In [46]:
myquery2 = 'nsa.sersic_mass > 3e11 AND nsa.z < 0.1'

In [47]:
q2 = Query(searchfilter=myquery2)
r2 = q2.run()

Your parsed filter is: 
and_(nsa.sersic_mass>3e11, nsa.z<0.1)
Results contain of a total of 3, only returning the first 3 results


In [68]:
print('total count', r2.totalcount)
r2.results

total count 3


[NamedTuple(mangaid=u'1-377378', plate=8132, name=u'6102', sersic_mass=513574535168.0, z=0.0265520866960287),
 NamedTuple(mangaid=u'1-338090', plate=8131, name=u'3701', sersic_mass=1150581604352.0, z=0.0806723535060883),
 NamedTuple(mangaid=u'1-256301', plate=8258, name=u'9102', sersic_mass=7116798033920.0, z=0.0961406528949738)]

## Compound Search Statements

We were hoping for a few more than 3 galaxies, so let's try to increase our search by broadening the criteria to also include galaxies with 127 fiber IFUs and a b/a ratio of at least 0.95.

To find 127 fiber IFUs, we'll use the **`name`** parameter of the **`ifu`** table, which means the full search parameter is **`ifu.name`**. However, **`ifu.name`** returns the IFU design name, such as `12701`, so we need to to set the value to `127*`.

The b/a ratio is in **`nsa`** table as the **`ba90`** parameter.

We're also going to join this to or previous query with an **`OR`** operator and use parentheses to group our individual search statements into a compound search statement.

In [58]:
myquery3 = 'nsa.sersic_mass > 3e11 AND (nsa.z < 0.1 OR ifu.name=127*) AND nsa.ba90 >= 0.95'

In [59]:
q3 = Query(searchfilter=myquery3)
r3 = q3.run()

Your parsed filter is: 
and_(nsa.sersic_mass>3e11, or_(nsa.z<0.1, ifu.name=127*), nsa.ba90>=0.95)
Results contain of a total of 1, only returning the first 1 results


In [60]:
print('totalcount', r3.totalcount)
r3.results

totalcount 1


[NamedTuple(mangaid=u'1-256301', plate=8258, name=u'9102', sersic_mass=7116798033920.0, ba90=0.96965616941452, z=0.0961406528949738)]

## Design Your Own Search

OK, now it's your turn to try designing a search.

**Exercise**: Write a search filter that will find galaxies with a redshift less than 0.02 that were observed with the 1901 IFU?

In [63]:
myquery = 'nsa.z < 0.02 and ifu.name == 1901'
q = Query(searchfilter=myquery)
r = q.run()
r.results

Your parsed filter is: 
and_(nsa.z<0.02, ifu.name==1901)
Results contain of a total of 8, only returning the first 8 results


[NamedTuple(mangaid=u'1-22438', plate=7992, name=u'1901', z=0.016383046284318),
 NamedTuple(mangaid=u'1-113520', plate=7815, name=u'1901', z=0.0167652331292629),
 NamedTuple(mangaid=u'1-113698', plate=8618, name=u'1901', z=0.0167444702237844),
 NamedTuple(mangaid=u'1-134004', plate=8486, name=u'1901', z=0.0185601413249969),
 NamedTuple(mangaid=u'1-155903', plate=8439, name=u'1901', z=0.0163660924881697),
 NamedTuple(mangaid=u'1-167079', plate=8459, name=u'1901', z=0.0157109703868628),
 NamedTuple(mangaid=u'1-209729', plate=8549, name=u'1901', z=0.0195561610162258),
 NamedTuple(mangaid=u'1-277339', plate=8254, name=u'1901', z=0.0192211158573627)]

You should get 8 results:

`[NamedTuple(mangaid='1-22438', plate=7992, name='1901', z=0.016383046284318),
 NamedTuple(mangaid='1-113520', plate=7815, name='1901', z=0.0167652331292629),
 NamedTuple(mangaid='1-113698', plate=8618, name='1901', z=0.0167444702237844),
 NamedTuple(mangaid='1-134004', plate=8486, name='1901', z=0.0185601413249969),
 NamedTuple(mangaid='1-155903', plate=8439, name='1901', z=0.0163660924881697),
 NamedTuple(mangaid='1-167079', plate=8459, name='1901', z=0.0157109703868628),
 NamedTuple(mangaid='1-209729', plate=8549, name='1901', z=0.0195561610162258),
 NamedTuple(mangaid='1-277339', plate=8254, name='1901', z=0.0192211158573627)]`

## Finding the Available Parameters
Now you might want to go out and try all of the interesting queries that you've been saving up, but you don't know what the parameters are called or what database table they are in. 

You can find all of the availabale parameters by:
1. clicking on in the **Return Parameters** dropdown menu on the left side of the **[Marvin-web Search page](https://sas.sdss.org/marvin2/search/)**,
2. reading the **[Marvin Docs page](https://api.sdss.org/doc/manga/marvin/refactor/query_params.html#marvin-query-parameters)**, or
3. via **Marvin-tools** (see next two cells)

In [45]:
# You might have to do an svn update to get this to work (otherwise try the next cell)
q = Query()
q.get_available_params()

[u'cube_header.cube',
 u'cube_header.header',
 u'maskbit.bit',
 u'maskbit.description',
 u'maskbit.flag',
 u'maskbit.label',
 u'maskbit_labels.flag',
 u'maskbit_labels.labels',
 u'maskbit_labels.maskbit',
 u'binid.binid',
 u'binid.index',
 u'binmode.name',
 u'binmode.structures',
 u'bintype.name',
 u'bintype.structures',
 u'current_default.file',
 u'current_default.filename',
 u'current_default.filepath',
 u'emline.file',
 u'emline.ivar',
 u'emline.mask',
 u'emline.parameter',
 u'emline.type',
 u'emline.value',
 u'emline_parameter.emlines',
 u'emline_parameter.name',
 u'emline_parameter.unit',
 u'emline_type.channel',
 u'emline_type.emlines',
 u'emline_type.name',
 u'emline_type.rest_wavelength',
 u'executionplan.comments',
 u'executionplan.id',
 u'executionplan.structures',
 u'extcol.hdus',
 u'extcol.name',
 u'extname.hdus',
 u'extname.name',
 u'exttype.hdus',
 u'exttype.name',
 u'file.cube',
 u'file.current_default',
 u'file.emlines',
 u'file.filename',
 u'file.filepath',
 u'file.fil

In [32]:
# try this if the previous cell didn't return a list of parameters
from marvin.api.api import Interaction
from pprint import pprint
url = config.urlmap['api']['getparams']['url']
ii = Interaction(route=url)
mykeys = ii.getData()
pprint(mykeys)

[u'cube_header.cube',
 u'cube_header.header',
 u'maskbit.bit',
 u'maskbit.description',
 u'maskbit.flag',
 u'maskbit.label',
 u'maskbit_labels.flag',
 u'maskbit_labels.labels',
 u'maskbit_labels.maskbit',
 u'binid.binid',
 u'binid.index',
 u'binmode.name',
 u'binmode.structures',
 u'bintype.name',
 u'bintype.structures',
 u'current_default.file',
 u'current_default.filename',
 u'current_default.filepath',
 u'emline.file',
 u'emline.ivar',
 u'emline.mask',
 u'emline.parameter',
 u'emline.type',
 u'emline.value',
 u'emline_parameter.emlines',
 u'emline_parameter.name',
 u'emline_parameter.unit',
 u'emline_type.channel',
 u'emline_type.emlines',
 u'emline_type.name',
 u'emline_type.rest_wavelength',
 u'executionplan.comments',
 u'executionplan.id',
 u'executionplan.structures',
 u'extcol.hdus',
 u'extcol.name',
 u'extname.hdus',
 u'extname.name',
 u'exttype.hdus',
 u'exttype.name',
 u'file.cube',
 u'file.current_default',
 u'file.emlines',
 u'file.filename',
 u'file.filepath',
 u'file.fil

Go ahead and try to create some new searches on your own from the parameter list. Please feel free to also try out the some of the same search on the **[Marvin-web Search page](https://sas.sdss.org/marvin2/search/)**.

## Returning Bonus Parameters

Often you want to run a query and see the value of parameters that you didn't explicitly search on. For instance, you want to find galaxies above a redshift of 0.1 and would like to know their RA and DECs.

In Marvin-tools, this is as easy as specifying the **`returnparams`** option with either a string (for a single bonus parameter) or a list of strings (for multiple bonus parameters).

In [69]:
myquery5 = 'nsa.z > 0.1'
bonusparams5 = ['cube.ra', 'cube.dec'] # table.parameter
# bonusparams5 = 'cube.ra'  # This works too

q5 = Query(searchfilter=myquery5, returnparams=bonusparams5)
r5 = q5.run()

Your parsed filter is: 
nsa.z>0.1
Results contain of a total of 103, only returning the first 103 results


In [70]:
r5.results

[NamedTuple(ra=253.40555905, dec=63.0312699156, mangaid=u'1-22301', plate=7992, name=u'6101', z=0.105152934789658),
 NamedTuple(ra=111.733682055, dec=41.0266910782, mangaid=u'1-548024', plate=8132, name=u'6101', z=0.129403337836266),
 NamedTuple(ra=180.172706776, dec=43.3546289934, mangaid=u'1-281234', plate=8259, name=u'12705', z=0.112710684537888),
 NamedTuple(ra=120.440525845, dec=44.1100506919, mangaid=u'1-44549', plate=8141, name=u'6104', z=0.131695568561554),
 NamedTuple(ra=133.035323408, dec=51.3127390765, mangaid=u'1-73638', plate=8244, name=u'12704', z=0.115137368440628),
 NamedTuple(ra=121.014200537, dec=40.8026129505, mangaid=u'1-72322', plate=8143, name=u'6101', z=0.126168444752693),
 NamedTuple(ra=169.669552898, dec=45.5527255595, mangaid=u'1-279123', plate=8466, name=u'6104', z=0.112603604793549),
 NamedTuple(ra=169.539016286, dec=45.6186388631, mangaid=u'1-279120', plate=8466, name=u'3703', z=0.111132927238941),
 NamedTuple(ra=168.094030871, dec=44.8160370649, mangaid=u'

Next time, we'll take a closer look at the **`Results`** class and its built in MaNGA convenience functions.

## Useful Resources

Check out these pages on the Marvin Docs site for more **`Query`** tips and tricks.

- **[Query](https://api.sdss.org/doc/manga/marvin/refactor/query.html)**
- **[Example Queries](https://api.sdss.org/doc/manga/marvin/refactor/tutorials/query-examples.html)**
- **[Results](https://api.sdss.org/doc/manga/marvin/refactor/results.htmlpages)** 
- **[Boolean Search Tutorial](https://api.sdss.org/doc/manga/marvin/refactor/tutorials/boolean-search-tutorial.html)**

In [71]:
print(r)

Results(results=[NamedTuple(mangaid=u'1-22438', plate=7992, name=u'1901', z=0.016383046284318), NamedTuple(mangaid=u'1-113520', plate=7815, name=u'1901', z=0.0167652331292629), NamedTuple(mangaid=u'1-113698', plate=8618, name=u'1901', z=0.0167444702237844), NamedTuple(mangaid=u'1-134004', plate=8486, name=u'1901', z=0.0185601413249969), NamedTuple(mangaid=u'1-155903', plate=8439, name=u'1901', z=0.0163660924881697)], 
query=u"SELECT mangadatadb.cube.mangaid, mangadatadb.cube.plate, mangadatadb.ifudesign.name, mangasampledb.nsa.z \nFROM mangadatadb.cube JOIN mangadatadb.ifudesign ON mangadatadb.ifudesign.pk = mangadatadb.cube.ifudesign_pk JOIN mangasampledb.manga_target ON mangasampledb.manga_target.pk = mangadatadb.cube.manga_target_pk JOIN mangasampledb.manga_target_to_nsa ON mangasampledb.manga_target.pk = mangasampledb.manga_target_to_nsa.manga_target_pk JOIN mangasampledb.nsa ON mangasampledb.nsa.pk = mangasampledb.manga_target_to_nsa.nsa_pk JOIN mangadatadb.pipeline_info AS drpali

In [73]:
r.totalcount, r.count

(8, 8)

In [75]:
r.showQuery()

u"SELECT mangadatadb.cube.mangaid, mangadatadb.cube.plate, mangadatadb.ifudesign.name, mangasampledb.nsa.z \nFROM mangadatadb.cube JOIN mangadatadb.ifudesign ON mangadatadb.ifudesign.pk = mangadatadb.cube.ifudesign_pk JOIN mangasampledb.manga_target ON mangasampledb.manga_target.pk = mangadatadb.cube.manga_target_pk JOIN mangasampledb.manga_target_to_nsa ON mangasampledb.manga_target.pk = mangasampledb.manga_target_to_nsa.manga_target_pk JOIN mangasampledb.nsa ON mangasampledb.nsa.pk = mangasampledb.manga_target_to_nsa.nsa_pk JOIN mangadatadb.pipeline_info AS drpalias ON drpalias.pk = mangadatadb.cube.pipeline_info_pk \nWHERE mangasampledb.nsa.z < 0.02 AND lower(mangadatadb.ifudesign.name) = lower('1901') AND drpalias.pk = 21"

In [82]:
r.toTable()

mangaid,plate,name,z
unicode8,int64,unicode4,float64
1-22438,7992,1901,0.0163830462843
1-113520,7815,1901,0.0167652331293
1-113698,8618,1901,0.0167444702238
1-134004,8486,1901,0.018560141325
1-155903,8439,1901,0.0163660924882
1-167079,8459,1901,0.0157109703869
1-209729,8549,1901,0.0195561610162
1-277339,8254,1901,0.0192211158574


In [83]:
r.toDataFrame()

,mangaid,plate,name,z
0,1-22438,7992,1901,0.016383
1,1-113520,7815,1901,0.016765
2,1-113698,8618,1901,0.016744
3,1-134004,8486,1901,0.018560
4,1-155903,8439,1901,0.016366
5,1-167079,8459,1901,0.015711
6,1-209729,8549,1901,0.019556
7,1-277339,8254,1901,0.019221


In [85]:
r.toJson()

'[["1-22438", 7992, "1901", 0.016383046284318], ["1-113520", 7815, "1901", 0.0167652331292629], ["1-113698", 8618, "1901", 0.0167444702237844], ["1-134004", 8486, "1901", 0.0185601413249969], ["1-155903", 8439, "1901", 0.0163660924881697], ["1-167079", 8459, "1901", 0.0157109703868628], ["1-209729", 8549, "1901", 0.0195561610162258], ["1-277339", 8254, "1901", 0.0192211158573627]]'

In [87]:
r.results

[NamedTuple(mangaid=u'1-22438', plate=7992, name=u'1901', z=0.016383046284318),
 NamedTuple(mangaid=u'1-113520', plate=7815, name=u'1901', z=0.0167652331292629),
 NamedTuple(mangaid=u'1-113698', plate=8618, name=u'1901', z=0.0167444702237844),
 NamedTuple(mangaid=u'1-134004', plate=8486, name=u'1901', z=0.0185601413249969),
 NamedTuple(mangaid=u'1-155903', plate=8439, name=u'1901', z=0.0163660924881697),
 NamedTuple(mangaid=u'1-167079', plate=8459, name=u'1901', z=0.0157109703868628),
 NamedTuple(mangaid=u'1-209729', plate=8549, name=u'1901', z=0.0195561610162258),
 NamedTuple(mangaid=u'1-277339', plate=8254, name=u'1901', z=0.0192211158573627)]

In [88]:
r.getColumns()

[u'mangaid', u'plate', u'name', u'z']

In [89]:
r.mapColumnsToParams()

['cube.mangaid', 'cube.plate', 'ifu.name', 'nsa.z']

In [90]:
r.getListOf('nsa.z')

[0.016383046284318,
 0.0167652331292629,
 0.0167444702237844,
 0.0185601413249969,
 0.0163660924881697,
 0.0157109703868628,
 0.0195561610162258,
 0.0192211158573627]

In [93]:
r.getDictOf(format_type='dictlist')

{'cube.mangaid': [u'1-22438',
  u'1-113520',
  u'1-113698',
  u'1-134004',
  u'1-155903',
  u'1-167079',
  u'1-209729',
  u'1-277339'],
 'cube.plate': [7992, 7815, 8618, 8486, 8439, 8459, 8549, 8254],
 'ifu.name': [u'1901',
  u'1901',
  u'1901',
  u'1901',
  u'1901',
  u'1901',
  u'1901',
  u'1901'],
 'nsa.z': [0.016383046284318,
  0.0167652331292629,
  0.0167444702237844,
  0.0185601413249969,
  0.0163660924881697,
  0.0157109703868628,
  0.0195561610162258,
  0.0192211158573627]}

In [94]:
r.results

[NamedTuple(mangaid=u'1-22438', plate=7992, name=u'1901', z=0.016383046284318),
 NamedTuple(mangaid=u'1-113520', plate=7815, name=u'1901', z=0.0167652331292629),
 NamedTuple(mangaid=u'1-113698', plate=8618, name=u'1901', z=0.0167444702237844),
 NamedTuple(mangaid=u'1-134004', plate=8486, name=u'1901', z=0.0185601413249969),
 NamedTuple(mangaid=u'1-155903', plate=8439, name=u'1901', z=0.0163660924881697),
 NamedTuple(mangaid=u'1-167079', plate=8459, name=u'1901', z=0.0157109703868628),
 NamedTuple(mangaid=u'1-209729', plate=8549, name=u'1901', z=0.0195561610162258),
 NamedTuple(mangaid=u'1-277339', plate=8254, name=u'1901', z=0.0192211158573627)]

In [95]:
r.convertToTool('cube')

Converting results to Marvin Cube objects


In [96]:
r.objects

[<Marvin Cube (plateifu='7992-1901', mode='remote', data_origin='api')>,
 <Marvin Cube (plateifu='7815-1901', mode='remote', data_origin='api')>,
 <Marvin Cube (plateifu='8618-1901', mode='remote', data_origin='api')>,
 <Marvin Cube (plateifu='8486-1901', mode='remote', data_origin='api')>,
 <Marvin Cube (plateifu='8439-1901', mode='remote', data_origin='api')>,
 <Marvin Cube (plateifu='8459-1901', mode='remote', data_origin='api')>,
 <Marvin Cube (plateifu='8549-1901', mode='remote', data_origin='api')>,
 <Marvin Cube (plateifu='8254-1901', mode='remote', data_origin='api')>]

In [97]:
r.objects[0]

<Marvin Cube (plateifu='7992-1901', mode='remote', data_origin='api')>

In [98]:
spax = r.objects[0][10,10]

In [99]:
spax

<Marvin Spaxel (x=10, y=10)>

In [101]:
spax.drp.plot()

In [102]:
r.download()

SDSS_ACCESS> syncing... please wait
SDSS_ACCESS> Done!


In [103]:
r.results

[NamedTuple(mangaid=u'1-22438', plate=7992, name=u'1901', z=0.016383046284318),
 NamedTuple(mangaid=u'1-113520', plate=7815, name=u'1901', z=0.0167652331292629),
 NamedTuple(mangaid=u'1-113698', plate=8618, name=u'1901', z=0.0167444702237844),
 NamedTuple(mangaid=u'1-134004', plate=8486, name=u'1901', z=0.0185601413249969),
 NamedTuple(mangaid=u'1-155903', plate=8439, name=u'1901', z=0.0163660924881697),
 NamedTuple(mangaid=u'1-167079', plate=8459, name=u'1901', z=0.0157109703868628),
 NamedTuple(mangaid=u'1-209729', plate=8549, name=u'1901', z=0.0195561610162258),
 NamedTuple(mangaid=u'1-277339', plate=8254, name=u'1901', z=0.0192211158573627)]

In [104]:
r.sort('nsa.z')

[NamedTuple(mangaid=u'1-167079', plate=8459, name=u'1901', z=0.0157109703868628),
 NamedTuple(mangaid=u'1-155903', plate=8439, name=u'1901', z=0.0163660924881697),
 NamedTuple(mangaid=u'1-22438', plate=7992, name=u'1901', z=0.016383046284318),
 NamedTuple(mangaid=u'1-113698', plate=8618, name=u'1901', z=0.0167444702237844),
 NamedTuple(mangaid=u'1-113520', plate=7815, name=u'1901', z=0.0167652331292629),
 NamedTuple(mangaid=u'1-134004', plate=8486, name=u'1901', z=0.0185601413249969),
 NamedTuple(mangaid=u'1-277339', plate=8254, name=u'1901', z=0.0192211158573627),
 NamedTuple(mangaid=u'1-209729', plate=8549, name=u'1901', z=0.0195561610162258)]

In [105]:
r.sort('mangaid')

[NamedTuple(mangaid=u'1-113520', plate=7815, name=u'1901', z=0.0167652331292629),
 NamedTuple(mangaid=u'1-113698', plate=8618, name=u'1901', z=0.0167444702237844),
 NamedTuple(mangaid=u'1-134004', plate=8486, name=u'1901', z=0.0185601413249969),
 NamedTuple(mangaid=u'1-155903', plate=8439, name=u'1901', z=0.0163660924881697),
 NamedTuple(mangaid=u'1-167079', plate=8459, name=u'1901', z=0.0157109703868628),
 NamedTuple(mangaid=u'1-209729', plate=8549, name=u'1901', z=0.0195561610162258),
 NamedTuple(mangaid=u'1-22438', plate=7992, name=u'1901', z=0.016383046284318),
 NamedTuple(mangaid=u'1-277339', plate=8254, name=u'1901', z=0.0192211158573627)]